In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

from sys import path
from os.path import abspath
path.append(abspath("/home/ng136/nico"))
import ng_functions as ng

from glob import glob
from pathlib import Path
import warnings
from time import time
from tqdm import tqdm

from datetime import datetime

# Load untreated non-CD101-sorted libraries

In [3]:
n_states = ['N1a', 'N1b', 'N2', 'N3', 'N4', 'N5', 'N6']

adata = sc.read('/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_neutrophils_untreated_annotated_embedding_5258x13126_backup_220422_15h46.h5ad')
adata = adata[(adata.obs['smoothed_Zilionis'].isin(n_states)) & (adata.obs['sample_condition'] != 'KP19_CD101')]

# Compute CLR expression

In [4]:
adtdata = sc.AnnData(adata.obs.loc[adata.obs['sample_condition'] != 'KP19_CD101',
                                   adata.obs.columns.str.contains('_Ab') & ~adata.obs.columns.isin(['CD101_Ab', 'Lair1_Ab'])])

adtdata.obs = adata.obs.loc[adata.obs['sample_condition'] != 'KP19_CD101',~adata.obs.columns.str.contains('_Ab')]
                            
adtdata.layers['clr'] = ng.clr_transform(adtdata.X)

# Inherit embedding from gene expression

In [6]:
adtdata.obsp = adata.obsp.copy()
adtdata.obsm = adata.obsm.copy()
adtdata.uns = adata.uns.copy()


# Save

In [7]:
start = time()
fname = '/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_neutrophils_untreated_adt_clr_no_cd101_{}x{}_backup_{}.h5ad'.format(*adtdata.shape,ng.now())
adtdata.write(fname)
print(fname)
ng.print_etime(start)

/n/groups/klein/nico/neutrophils/backups/totalseq_exp2_neutrophils_untreated_adt_clr_no_cd101_3978x20_backup_220601_04h57.h5ad
Elapsed time: 0 minutes and 0.1 seconds.
